# Phase 5: Run Interactions

Documentation: https://halllab.atlassian.net/wiki/spaces/IGEM/pages/84705288/Phase+5+Run+Regression

Points to check (with Nikki):
- Run skewness only to Phenotypes?
- Why she run the Survey to ECHO EAWS Analysis?

In [113]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import igem
import clarite

### Read Source Data

In [3]:
# define the path to the data folder
path = Path().resolve()
path_data = path / "data"

In [4]:
# Read NHANES Data (with medications normalized to LDL and TC)
df_nhanes = igem.epc.load.from_csv(
    str(path_data) +  "/step_04_02_nhanes_data_with_medications.csv",
    index_col='ID'
    )
df_nhanes.index = df_nhanes.index.astype(int)
print(f"Start Run Interactions Process with: {len(df_nhanes)} records")

Loaded 55,206 observations of 918 variables
Start Run Interactions Process with: 55206 records


/Users/andrerico/Works/Projects/pbs_igem/.venv/lib/python3.10/site-packages/igem/epc/clarite/load/load.py:77: DtypeWarning: Columns (442,869) have mixed types.Specify dtype option on import or set low_memory=False.
  return clarite.load.from_csv(filename, index_col, **kwargs)


In [5]:
# Read Moldel (we need to clean interactions that is not in coluns list)
df_model = igem.epc.load.from_csv(str(path_data) + "/step_01_05_Models.csv") 
print(f"Start with: {len(df_model)} interactions")

Loaded 992,923 observations of 5 variables
Start with: 992923 interactions


### Session for processing identified interaction data (GE.db)

In [6]:
# Keep only interactions terms
df_model = igem.epc.modify.colfilter(
    df_model,
    only=['field_name_1', 'field_name_2']
    )

Running colfilter
--------------------------------------------------------------------------------
Keeping 2 of 5 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 0 continuous variables
	2 of 5 unknown variables


Note: A classification was performed to identify the fields in NHANES that were exposure factors, and we used these selected fields to search for relationships in IGEM.

With Nikki joining the project, she helped refine this list of exposure factors using the data filtered in GE.filter. The search for data in NHANES utilized the list of relevant exposures evaluated by Nikki, which is more restricted.

Therefore, the code below will retain interactions only if both terms are present in df_nhanes.

In [7]:
# Keep only interactions that are in the columns of df_nhanes
# Create a set of df_nhanes columns for quick checking
nhanes_columns = set(df_nhanes.columns)
# filter df_models to keep only interactions where both terms are in df_nhanes columns
df_models_filtered = df_model[df_model.apply(lambda row: row['field_name_1'] in nhanes_columns and row['field_name_2'] in nhanes_columns, axis=1)]
print(f"Now has {len(df_models_filtered)} interactions after filtering")
df_models_filtered.reset_index(drop=True, inplace=True)

Now has 213059 interactions after filtering


### Split Cohorts to LDL, TC and Triglycerides

LDL, Total-C and Triglycerides split in Discovery (1999-2008) and Replicate (2009-2018)

Note: The fields for LDL and Total Cholesterol have been adjusted to account for participants who use any medication to control cholesterol levels. This adjustment is documented in Phase 4.

In [8]:
list_covariates = ['RIDAGEYR', 'RIAGENDR', 'RIDRETH1', 'BMXBMI', 'Cycle']
list_phenotypes_wo_adj = ['LBDLDL', 'LBXTC']
list_phenotypes = ['LBDLDL_N', 'LBXTC_N', 'LBXSTR', 'LBDHDL', 'LBXHDD', 'LBDHDD']

# Define the list of exposes
excluded_columns = set(list_covariates + list_phenotypes + list_phenotypes_wo_adj)
list_exposes = [col for col in df_nhanes.columns if col not in excluded_columns]

In [9]:
# Function to check if columns exist in DataFrame
def columns_exist(df, cols):
    return all(col in df.columns for col in cols)

In [10]:
# DataFrame to collect results
df_results_discover_final = pd.DataFrame()
df_results_replicate_final = pd.DataFrame()
list_results_discover = []
list_results_replicate = []

#### Run ExE Interaction Analysis to LDL Phenotype

Defines Discovery and Replicate groups based on cycles.
Aligns both groups to have the same exposure factors.

In [11]:
# Note: In production, this block is inside the phenotype loop.
list_phenotypes = ['LBDLDL_N',]
for i_outcome in list_phenotypes:
    print(f"Start with: {i_outcome}")


Start with: LBDLDL_N


In [ ]:
# # QC on df_nhanes

# # drop all object type columns
# object_columns = df_nhanes.select_dtypes(include=['object']).columns
# df_nhanes = df_nhanes.drop(columns=object_columns)

# # drop all row with any NAN
# df_maintable_exe = df_nhanes.dropna()

# # drop all constants rows
# non_constant_columns = df_maintable_exe.columns[df_maintable_exe.nunique() > 1]
# df_maintable_exe = df_maintable_exe[non_constant_columns]



Note:  The sequence of instructions below is part of the loop initiated above. 
       For development and debugging purposes, the chain has been broken down.

The following code divides the NHANES data into two groups: discovery and replication. 

The discovery group consists of data from earlier cycles, while the replication group consists of data from later cycles.

For both df_nhanes_discovery and df_nhanes_replicate, the Cycle column is converted to a categorical type with a specified order.

Skip if the outcome is HDL-c / This will split in 3 cohorts

Check: https://halllab.atlassian.net/wiki/spaces/IGEM/pages/62619672/Different+HDL+Cholesterol+Codes+in+NHANES+LBDHDL+LBXHDD+LBDHDD

In [159]:
if i_outcome in ['LBDHDL', 'LBXHDD', 'LBDHDD']:
    ...
else:
    # Define discovery and replication cycles
    cycles_discovery = ['1999-2000', '2001-2002', '2003-2004', '2005-2006', '2007-2008']
    cycles_replicate = ['2009-2010', '2011-2012', '2013-2014', '2015-2016', '2017-2018']

    # Filter data for discovery and replication groups
    df_nhanes_discovery = df_nhanes[df_nhanes['Cycle'].isin(cycles_discovery)]
    df_nhanes_replicate = df_nhanes[df_nhanes['Cycle'].isin(cycles_replicate)]

    # Set the oder of cycles
    df_nhanes_discovery['Cycle'] = pd.Categorical(df_nhanes_discovery['Cycle'], categories=cycles_discovery, ordered=True)
    df_nhanes_replicate['Cycle'] = pd.Categorical(df_nhanes_replicate['Cycle'], categories=cycles_replicate, ordered=True)

In [160]:
# Filter the DataFrames to keep only the columns of interest
df_nhanes_discovery = df_nhanes_discovery[[i_outcome] + list_covariates + list_exposes].dropna(subset=[i_outcome])
df_nhanes_replicate = df_nhanes_replicate[[i_outcome] + list_covariates + list_exposes].dropna(subset=[i_outcome])

# Sync both DataFrames to have the same Exposure columns
# Drop columns with all NaN values in both DataFrames
df_nhanes_discovery = df_nhanes_discovery.dropna(axis=1, how='all')
df_nhanes_replicate = df_nhanes_replicate.dropna(axis=1, how='all')

# get the common columns
common_columns = df_nhanes_discovery.columns.intersection(df_nhanes_replicate.columns)

# filter both DataFrames to keep only the common columns
df_nhanes_discovery = df_nhanes_discovery[common_columns]
df_nhanes_replicate = df_nhanes_replicate[common_columns]

In [161]:
# check if both groups as the same number of columns
n_discovery_exe = len(df_nhanes_discovery.columns)
n_replicate_exe = len(df_nhanes_replicate.columns)
if n_discovery_exe != n_replicate_exe:
    # Raise an error if the number of columns is different
    print(f"Discovery has {n_discovery_exe} columns and Replicate has {n_replicate_exe} columns")
    print("Columns in Discovery but not in Replicate:")
    print(set(df_nhanes_discovery.columns) - set(df_nhanes_replicate.columns))
    print("Columns in Replicate but not in Discovery:")
    print(set(df_nhanes_replicate.columns) - set(df_nhanes_discovery.columns))
    print("Skipping this outcome")
    # continue
    ...
print(f"{n_discovery_exe} columns and {len(df_nhanes_discovery)} rows on discovery dataset")
print(f"{n_replicate_exe} columns and {len(df_nhanes_replicate)} rows on replicate dataset")


301 columns and 11453 rows on discovery dataset
301 columns and 12695 rows on replicate dataset


In [162]:
# # Categories the columns types
df_nhanes_discovery = igem.epc.modify.categorize(df_nhanes_discovery)
df_nhanes_replicate = igem.epc.modify.categorize(df_nhanes_replicate)

Running categorize
--------------------------------------------------------------------------------
43 of 301 variables (14.29%) are classified as constant (1 unique value).
76 of 301 variables (25.25%) are classified as binary (2 unique values).
55 of 301 variables (18.27%) are classified as categorical (3 to 6 unique values).
116 of 301 variables (38.54%) are classified as continuous (>= 15 unique values).
0 of 301 variables (0.00%) were dropped.
11 of 301 variables (3.65%) were not categorized and need to be set manually.
	10 variables had between 6 and 15 unique values
	1 variables had >= 15 values but couldn't be converted to continuous (numeric) values
Running categorize
--------------------------------------------------------------------------------
41 of 301 variables (13.62%) are classified as constant (1 unique value).
89 of 301 variables (29.57%) are classified as binary (2 unique values).
39 of 301 variables (12.96%) are classified as categorical (3 to 6 unique values).
116

Debug propose: Select two expose factores. 

In [163]:
e1 = "ALQ140Q"
e2 = "ALQ150"

# Check if columns exist in the DataFrame
if columns_exist(df_nhanes_discovery, [e1, e2]):
    # create a DataFrame with the columns of interest
    df_maintable_exe = df_nhanes_discovery.loc[:, list_covariates + [i_outcome, e1, e2]]
    print(f"Processed with: {e1} and {e2}")
else:
    print(f"Skipped: {e1} and/or {e2} not found")

print(len(df_maintable_exe))
print(e1, " - ", e2)
print(i_outcome)
print(list_covariates)
print(df_maintable_exe.dtypes)
print(df_maintable_exe.head())

Processed with: ALQ140Q and ALQ150
11453
ALQ140Q  -  ALQ150
LBDLDL_N
['RIDAGEYR', 'RIAGENDR', 'RIDRETH1', 'BMXBMI', 'Cycle']
RIDAGEYR     float64
RIAGENDR    category
RIDRETH1    category
BMXBMI       float64
Cycle       category
LBDLDL_N     float64
ALQ140Q      float64
ALQ150      category
dtype: object
       RIDAGEYR RIAGENDR RIDRETH1  BMXBMI      Cycle  LBDLDL_N  ALQ140Q ALQ150
ID                                                                            
41479      52.0      1.0      1.0   27.56  2007-2008     121.0      3.0    2.0
41485      30.0      2.0      2.0   25.99  2007-2008     119.0      NaN    NaN
41486      61.0      2.0      1.0   31.21  2007-2008     110.0      NaN    NaN
41487      27.0      1.0      5.0   23.44  2007-2008     105.0      0.0    2.0
41489      40.0      2.0      1.0   36.59  2007-2008     106.0      3.0    2.0


In [36]:
Interation_Study = igem.epc.analyze.interaction_study(
        data=df_maintable_exe,
        outcomes=i_outcome,
        interactions=[(e1, e2)],
        covariates=list_covariates,
    )


InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'LBDLDL_N'
Using 11,453 of 11,453 observations
Regressing 2 variables
	0 binary variables
	1 categorical variables
	1 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 14 processes...

	Finished Running 1 interactions
0 tests had an error
Completed Interaction Study for LBDLDL_N

Completed association study


### Create the loop to process all the data.

In [18]:
df_models_filtered

,field_name_1,field_name_2
0,LBDLG1LC,ARQ077
1,LBDLG1LC,ARQ034D
2,LBXLG1,ARQ077
3,LBXLG1,ARQ034D
4,SSLG1_N,ARQ077
...,...,...
213054,LBDSZNSI,SMD830
213055,LBDSZNSI,SMD770
213056,LBDSZNSI,SMD800
213057,LBDSZNSI,SMD740


In [165]:
# variables to control
less_t = 0
no_col = 0
process = 0
no_exp = 0

for i_mappair in df_models_filtered.index:
    # get Exposomes
    e1 = df_models_filtered["field_name_1"][i_mappair]
    e2 = df_models_filtered["field_name_2"][i_mappair]
    

    # Check if columns exist in the DataFrame
    if columns_exist(df_nhanes_discovery, [e1, e2]):
        # create a DataFrame with the columns of interest
        # df_maintable_exe = df_nhanes_discovery.loc[:, list_covariates + [i_outcome, e1, e2]]
        
        v_list = list_covariates + [i_outcome, e1, e2]
        df_maintable_exe = igem.epc.modify.colfilter(df_nhanes_discovery, only=v_list)
        
        # Selecionando as colunas do tipo 'object'
        object_columns = df_maintable_exe.select_dtypes(include=['object']).columns

        # Removendo as colunas do tipo 'object'
        df_maintable_exe = df_maintable_exe.drop(columns=object_columns)


        # Drop all row with any NAN
        df_maintable_exe = df_maintable_exe.dropna()



        # df_maintable_exe = igem.epc.modify.categorize(df_maintable_exe)
        non_constant_columns = df_maintable_exe.columns[df_maintable_exe.nunique() > 1]
        df_maintable_exe = df_maintable_exe[non_constant_columns]


        if len(df_maintable_exe) < 200:
            # print(f"Skipped: {e1} and/or {e2} less than the threshold")
            less_t += 1
            continue
        if not columns_exist(df_maintable_exe, v_list):
            no_col += 1
            # print(f"Skipped: {e1} and {e2} no columns after drop")
            continue
        if e1 == e2:
            continue


        process += 1
        print(f"Processed with: {e1} and {e2}")
        # Run the interaction study
        Interation_Study = igem.epc.analyze.interaction_study(
            data=df_maintable_exe,
            outcomes=i_outcome,
            interactions=[(e1, e2)],
            covariates=list_covariates,
            min_n=200,
        )

        list_results_discover.append(
            [
                Interation_Study.LRT_pvalue.index.levels[2][0],
                Interation_Study.LRT_pvalue.index.levels[0][0],
                Interation_Study.LRT_pvalue.index.levels[1][0],
                Interation_Study.Converged.values[0],
                Interation_Study.LRT_pvalue.values[0],
                Interation_Study.LRT_pvalue.values[0] * len(df_maintable_exe),
            ]
        )
    else:
        # print(f"Skipped: {e1} and/or {e2} exposomes not found")
        no_exp += 1

# Create a DataFrame with the results
df_results_discover = pd.DataFrame(
    list_results_discover,
    columns=[
        "Outcome", "Term1", "Term2", "Converged", "LRT_pvalue", "Bonfp"
        ],
)

print(f"No exposomes: {no_exp}")
print(f"No data: {less_t}")
print(f"No columns: {no_col}")
print(f"Processed: {process}")

Running colfilter
--------------------------------------------------------------------------------
Keeping 8 of 301 variables:
	2 of 76 binary variables
	2 of 55 categorical variables
	4 of 116 continuous variables
	0 of 11 unknown variables
Running colfilter
--------------------------------------------------------------------------------
Keeping 8 of 301 variables:
	1 of 76 binary variables
	2 of 55 categorical variables
	5 of 116 continuous variables
	0 of 11 unknown variables
Processed with: LBXPFOA and RDD040

	Cycle missing '1999-2000', '2001-2002'
InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'LBDLDL_N'
Using 205 of 205 observations
Regressing 2 variables
	0 binary variables
	0 categorical variables
	2 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 14 processes...

	Finished Running 1 interactions
0 tests had an error
Completed Interaction Study for LBDLDL_N

Co

KeyboardInterrupt: 

In [146]:
df_results_discover = pd.DataFrame(
    list_results_discover,
    columns=[
        "Outcome", "Term1", "Term2", "Converged", "LRT_pvalue", "Bonfp"
        ],
)

In [147]:
df_results_discover

,Outcome,Term1,Term2,Converged,LRT_pvalue,Bonfp
0,LBDLDL_N,LBXPFOA,RDD040,True,0.679824,139.363819
1,LBDLDL_N,LBXPFOA,RDD040,True,0.679824,139.363819
2,LBDLDL_N,LBXPFOA,RDD040,True,0.679824,139.363819
3,LBDLDL_N,LBXPFUA,RDD040,True,0.377987,4329.084887
4,LBDLDL_N,LBDPFOAL,RDD040,True,NaN,NaN
...,...,...,...,...,...,...
7775,LBDLDL_N,LBDVXYLC,LBXTHG,True,0.096372,303.281704
7776,LBDLDL_N,LBDVXYLC,LBDTHGLC,True,0.640305,1471.421844
7777,LBDLDL_N,LBDVXYLC,URXUHG,True,0.802085,1026.669216
7778,LBDLDL_N,LBDVXYLC,URDUHGLC,True,NaN,NaN


In [142]:
df_maintable_exe
if not columns_exist(df_maintable_exe, v_list):
    print("error")
print(v_list)
df_maintable_exe.dtypes

if e1 == e2:
    print("igaus")

['RIDAGEYR', 'RIAGENDR', 'RIDRETH1', 'BMXBMI', 'Cycle', 'LBDLDL_N', 'LBXSGL', 'LBXSGL']
igaus


In [136]:
e1 = "RDD040"
e2 = "SMD100BR"
v_list = list_covariates + [i_outcome, e1, e2]
df_maintable_exe = igem.epc.modify.colfilter(df_nhanes_discovery, only=v_list)

# # # Drop all row with any NAN
df_maintable_exe = df_maintable_exe.dropna()

# df_maintable_exe = igem.epc.modify.categorize(df_maintable_exe)
non_constant_columns = df_maintable_exe.columns[df_maintable_exe.nunique() > 1]

# # Criando um novo DataFrame apenas com as colunas não constantes
df_maintable_exe = df_maintable_exe[non_constant_columns]


# Selecionando as colunas do tipo 'object'
object_columns = df_maintable_exe.select_dtypes(include=['object']).columns

# Removendo as colunas do tipo 'object'
df_maintable_exe = df_maintable_exe.drop(columns=object_columns)

# if len(df_maintable_exe) < 200:
#     print(f"Skipped: {e1} and/or {e2} less than the threshold")
#     less_t += 1
if not columns_exist(df_maintable_exe, v_list):
    no_col += 1
    print(f"Skipped: {e1} and {e2} no columns after drop")
else:
    print(df_maintable_exe)

df_maintable_exe

Running colfilter
--------------------------------------------------------------------------------
Keeping 8 of 301 variables:
	1 of 76 binary variables
	2 of 55 categorical variables
	4 of 116 continuous variables
	1 of 11 unknown variables
Skipped: RDD040 and SMD100BR no columns after drop


,LBDLDL_N,RIDAGEYR,RIAGENDR,RIDRETH1,BMXBMI,Cycle,RDD040
ID,,,,,,,
41489,106.000000,40.0,2.0,1.0,36.59,2007-2008,1.0
41617,108.571429,80.0,2.0,3.0,23.74,2007-2008,1.0
41654,212.000000,80.0,2.0,3.0,24.56,2007-2008,2.0
41666,191.000000,43.0,2.0,3.0,32.41,2007-2008,2.0
41688,181.000000,49.0,1.0,3.0,29.28,2007-2008,32.0
...,...,...,...,...,...,...,...
20849,116.000000,40.0,1.0,3.0,20.38,2001-2002,15.0
20851,160.000000,63.0,2.0,3.0,22.63,2001-2002,15.0
20884,102.000000,82.0,2.0,3.0,25.16,2001-2002,7.0


In [135]:
teste = igem.epc.modify.make_categorical(df_maintable_exe, only='SMD100BR')
teste.SMD100BR.dtypes

Running make_categorical
--------------------------------------------------------------------------------
Set 1 of 8 variable(s) as categorical, each with 709 observations


CategoricalDtype(categories=['1ST CLASS LIGHT', '305'S', 'BASIC', 'BASIC LIGHT',
                  'BASIC ULTRA LIGHT', 'BELAIR', 'BENSON & HEDGES',
                  'BENSON & HEDGES LIGHT', 'BENSON & HEDGES ULTRA LIGHT',
                  'BERKLEY',
                  ...
                  'USA GOLD LIGHT', 'VICEROY LIGHT', 'VIRGINIA SLIMS LIGHT',
                  'VIRGINIA SLIMS LUXURY ULTRA LIGHT',
                  'VIRGINIA SLIMS SUPERSLIMS', 'WINNER', 'WINSTON',
                  'WINSTON LIGHT', 'ZIG ZAG', 'nan'],
, ordered=False)

In [121]:
Interation_Study = clarite.analyze.interaction_study(
        data=df_maintable_exe,
        outcomes=i_outcome,
        interactions=[(e1, e2)],
        covariates=list_covariates,
        min_n=40
    )


	RIDRETH1 missing '2.0'
	Cycle missing '2003-2004', '1999-2000', '2001-2002'


ValueError: No variables are available to run regression on

In [60]:
Interation_Study

,,,Parameter,Converged,N,LRT_pvalue,Red_Var1_beta,Red_Var1_SE,Red_Var1_Pval,Red_Var2_beta,Red_Var2_SE,Red_Var2_Pval,Full_Var1_Var2_beta,Full_Var1_Var2_SE,Full_Var1_Var2_Pval,Full_Var1_beta,Full_Var1_SE,Full_Var1_Pval,Full_Var2_beta,Full_Var2_SE,Full_Var2_Pval,Log
Term1,Term2,Outcome,,,,,,,,,,,,,,,,,,,,
LBXPFUA,RDD040,LBDLDL_N,LBXPFUA:RDD040,True,205,0.377987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
